# 🚨 DDL: Tabela Silver - `tb_custo_chamado_invalidos` (Quarentena)

**Origem:** Validação Financeira sobre `custos`

## ⚙️ Regras de Qualidade

Captura transações de custo que não podem ser processadas contabilmente.

### Flags de Rejeição
- **`flag_valor_valido`**: Verifica se `vl_custo` é numérico e maior que zero (assumindo que custo não pode ser negativo nesta regra de negócio).
- **`flag_chamado_valido`**: Garante integridade referencial mínima (tem chamado atrelado?).
- **`flag_id_valido`**: Chave primária não nula.

### Constraints
- **Primary Key:** `pk_custo_invalidos` (`cd_custo`).

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_custo_chamado_invalidos"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_custo BIGINT COMMENT 'Identificador do registro de custo.',
    cd_chamado BIGINT COMMENT 'Vínculo com o chamado.',
    vl_custo DECIMAL(12,10) COMMENT 'Valor monetário do custo.',
    dt_ingestion TIMESTAMP COMMENT 'Data de ingestão do registro.',
    dc_origem STRING COMMENT 'Origem do dado (ex: supabase).',
    flag_id_valido BOOLEAN COMMENT 'Indica se o ID do custo é válido.',
    flag_chamado_valido BOOLEAN COMMENT 'Indica se o ID do chamado é válido.',
    flag_valor_valido BOOLEAN COMMENT 'Indica se o valor do custo é válido (> 0 e numérico).',
    flag_qualidade STRING COMMENT 'Status de qualidade: OK ou ERRO.',
    CONSTRAINT pk_custo_invalidos PRIMARY KEY (cd_custo)
)
COMMENT 'Tabela silver de auditoria - registros de custos rejeitados por problemas de qualidade';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")